# Welcome to your Quantitative Social Sciences Analysis Toolkit!



## 1. load survey data into the notebook 
(run this first & run this everytime you close and reopen the notebook)

In [26]:
import pandas as pd # load a specialized piece of software that will help us with the analysis
data = pd.read_csv('data/anes_pilot_2016.csv',low_memory=False) # read in the table of data
data = data[data.V161002 != -1] # remove any responent that wasn't interviewed face-to-face

## 2. display data

In [27]:
data.T # display a snapshot of raw data -- the first column here shows your variables, 
       # the other colums are responses

,0,1,2,3,4,5,6,7,8,9,...,1170,1171,1172,1173,1174,1175,1176,1177,1178,1179
Unnamed: 0,0,1,2,3,4,5,6,7,8,9,...,1170,1171,1172,1173,1174,1175,1176,1177,1178,1179
version,ANES2016TimeSeries_20190904,ANES2016TimeSeries_20190904,ANES2016TimeSeries_20190904,ANES2016TimeSeries_20190904,ANES2016TimeSeries_20190904,ANES2016TimeSeries_20190904,ANES2016TimeSeries_20190904,ANES2016TimeSeries_20190904,ANES2016TimeSeries_20190904,ANES2016TimeSeries_20190904,...,ANES2016TimeSeries_20190904,ANES2016TimeSeries_20190904,ANES2016TimeSeries_20190904,ANES2016TimeSeries_20190904,ANES2016TimeSeries_20190904,ANES2016TimeSeries_20190904,ANES2016TimeSeries_20190904,ANES2016TimeSeries_20190904,ANES2016TimeSeries_20190904,ANES2016TimeSeries_20190904
V160001,1,2,3,4,5,6,7,8,9,10,...,1171,1172,1173,1174,1175,1176,1177,1178,1179,1180
V160001_orig,300001,300002,300003,300004,300006,300007,300008,300012,300018,300020,...,302866,302867,302871,302872,302873,302874,302876,302877,302878,302880
V160101,0.827,1.0806,0.3878,0.3596,0.647,0.7062,3.9604,0.962,0.9761,0.6182,...,1.4991,0.7705,0.4732,0.6149,1.3355,1.4078,1.0449,1.323,0.552,1.4148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
V168524,81,82,82,82,82,81,81,81,82,82,...,82,82,81,82,81,82,-1,21,82,81
V168525,-1. Inap,-1. Inap,-1. Inap,-1. Inap,-1. Inap,-1. Inap,-1. Inap,-1. Inap,-1. Inap,-1. Inap,...,-1. Inap,-1. Inap,-1. Inap,-1. Inap,-1. Inap,-1. Inap,-1. Inap,Roy Asberry Cooper,-1. Inap,-1. Inap
V168526,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,1,-1,-1
V168527,-1. Inap,-1. Inap,-1. Inap,-1. Inap,-1. Inap,-1. Inap,-1. Inap,-1. Inap,-1. Inap,-1. Inap,...,-1. Inap,-1. Inap,-1. Inap,-1. Inap,-1. Inap,-1. Inap,-1. Inap,Patrick L. 'Pat' McCrory,-1. Inap,-1. Inap


## 3. get category counts for a categorical variable


V161002 = gender  
V161027 = for whom did respondent vote?

In [53]:
from prettytable import PrettyTable
from ipywidgets import interact
from IPython.core.display import display, HTML

@interact(variable=data.columns.sort_values())
def categorical_table(variable='V161002'):
    x = PrettyTable()
    x.field_names = [variable, 'Count']
    for i, row in data.groupby(variable).size().reset_index().iterrows():
        x.add_row((row[variable], row[0]))
    display(HTML(x.get_html_string()))

interactive(children=(Dropdown(description='variable', index=18, options=('Unnamed: 0', 'V160001', 'V160001_or…

## 4. get average and spread for a continuous variable

V161267 = age

In [52]:
from prettytable import PrettyTable
from ipywidgets import interact
import numpy as np
from IPython.core.display import display, HTML


@interact(variable=data.columns.sort_values())
def categorical_table(variable='V161267'):
    x = PrettyTable()
    x.field_names = [variable, 'mean', 'standard deviation']
    
    result = (variable, np.mean(data[variable]), np.std(data[variable]))
    x.add_row(result)
    display(HTML(x.get_html_string()))

interactive(children=(Dropdown(description='variable', index=367, options=('Unnamed: 0', 'V160001', 'V160001_o…